### Introduction to MCP

This notebook file introduces the concept of Model Context Protocol using OpenAI Agents SDK framework.

In [ ]:
# Importing required libraries

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [ ]:
load_dotenv(override=True)

#### Using MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [ ]:
fetch_params = {
    "command": "uvx", 
    "args": ["mcp-server-fetch"]
    }

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

#### Introducing another MCP server - this time Javascript based, with node

In [ ]:
# setting up path to node
env = {"PATH": "/usr/bin/node" + os.environ["PATH"]}
playwright_params = {
    "command": "npx",
    "args": [ "@playwright/mcp@latest"], 
    "env": env
    }

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
env = {"PATH": "/usr/bin/node" + os.environ["PATH"]}
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path], "env": env}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

#### Introducing Agent with Tools!

In [ ]:
env = {"PATH": "/usr/bin/node" + os.environ["PATH"]}
playwright_params = {
    "command": "npx",
    "args": [ "@playwright/mcp@latest"], 
    "env": env
    }

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
env = {"PATH": "/usr/bin/node" + os.environ["PATH"]}
files_params = {
    "command": "npx", 
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path], 
    "env": env
    }

In [ ]:
from agents.extensions.models.litellm_model import LitellmModel
import os

In [ ]:
api_key = os.getenv("AMAZON_BEDROCK_API_KEY")

In [ ]:
model_name = "us.amazon.nova-micro-v1:0"
model=LitellmModel(model=model_name, api_key=api_key)

In [ ]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=120) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=120) as mcp_server_browser:
        agent = Agent(
            name="mcp_researcher",
            instructions=instructions,
            model=model,
            mcp_servers=[mcp_server_browser, mcp_server_files]
        )

        with trace("mcp_researcher"):
            result = await Runner.run(agent, "Find the best way to present a LinkedIn profile to ensure recruiters are impressed and want to reach out. Present results as a markdown in linkedin.md")
            print(result.final_output)
